# X.0 A very brief intro to MNE



MNE is an open-source software package to help you analyze and visualize M/EEG, ECoG, and other types of neurophysiological data. Things you can do in MNE include:

* visualizing data
* epoching
* averaging and computing evoked responses
* computing ICA 
* time-frequency analysis
* computing connectivity estimates
* classifier analysis 
* non-parametric stats

...and many more. There's a wealth of documentation (https://martinos.org/mne/stable/documentation.html) and examples (https://martinos.org/mne/stable/auto_examples/index.html) online. 

In today's session, we'll briefly walk you through one of the MNE introductory examples so you can get a general feel for how to use MNE. 

Here's the original example that we'll be using: https://martinos.org/mne/stable/auto_tutorials/plot_introduction.html. We'll plot evoked responses today, but the original example also shows you how to run and plot a time-frequency analysis.

## X.1 Installing MNE

Downloading and installing MNE can take a little time, so for the purpose of today's session (or if you're generally not that interested in analyzing M/EEG data), we'd advise you to just follow along with the lecture from here on out, rather than downloading and installing the package and example data set.

Installation instructions can be found here: https://martinos.org/mne/stable/install_mne_python.html#install-python-and-mne-python. 


## X.2 Reading in raw data

Once you have MNE installed, you can already read in your raw data! MNE supports several different data formats, including fif, brainvision, eeglab, and edf files. You can read more about that here: https://martinos.org/mne/stable/manual/io.html

The example we're using works with MNE's sample data set, which comes as fif.

In [ ]:
import mne

# import the sample data set
from mne.datasets import sample

# specify the name of your raw data file
data_path = sample.data_path()
raw_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'
print(raw_fname)

# read in the data
raw = mne.io.read_raw_fif(raw_fname)

Let's have a look at the data structure:

In [ ]:
print(raw)

In [ ]:
print(raw.info)

This gives us a lot of useful information about our raw data: We can see that there are 376 channels (chs and nchan), what they're called (ch_names), the events (0 at this point, because we haven't defined them yet), channels marked as bad (MEG 2443 and EEG 053, apparently), highpass and lowpass filters that have been applied (0.1 Hz and 40 Hz), the sampling frequency (sfreq), and a lot more. We can look at each of these in more detail:

In [ ]:
# for example, look at the channel names
print(raw.ch_names)

In [ ]:
# or look at the bad channels
print(raw.info['bads'])

## X.3 Browsing raw data

Now let's have a look at a segment of our data:

In [ ]:
'''Note that usually, when using MNE outside of jupyter notebooks, 
this will open an interactive window where you can scroll through your data 
and, for example, mark bad channels manually.

For now, we'll just show a preview of what that looks like.'''

raw.plot();

You can read more about how to browse raw data in MNE here: https://martinos.org/mne/stable/manual/gui/browse.html#ch-browse

## X.4 Epoching data

MNE can extract events for you like this:

In [ ]:
# use function .find_events on the raw data
# and tell MNE that your stimulus channel is called 'STI 014'
events = mne.find_events(raw, stim_channel='STI 014')

# look at the first five events
print(events[:5])

If you'd like to exclude some noisy channels, you can save all the bad channels in a list, pass that list to MNE, and select only the remaining good channels, like this:

In [ ]:
raw.info['bads'] += ['MEG 2443', 'EEG 053']
picks = mne.pick_types(raw.info, meg=True, eeg=True, eog=True, stim=False,
                       exclude='bads')

Now the data frame 'picks' includes all the MEG, EEG and EOG channels, excluding the ones that are in your list of 'bad' channels and the STIM channels.

In [ ]:
# define parameters for epoching

event_id = dict(aud_l=1, aud_r=2)  # event trigger and conditions
tmin = -0.2  # start of each epoch (200ms before the trigger)
tmax = 0.5  # end of each epoch (500ms after the trigger)
baseline = (None, 0)  # means from the first instant to t = 0
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6) # set criteria for automatic artifact rejection

# epoch your data
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                    baseline=baseline, preload=False, reject=reject)
print(epochs)

As you can see above, the epochs data frame now includes two conditions: 'aud_l' (which corresponded to event_id 1), with 72 epochs, and 'aud_r' (event_id 2), with 73 epochs. You can compute evoked responses as follows:

In [ ]:
# subset epochs that were from condition 'aud_l' 
# and average across all epochs
evoked_left = epochs['aud_l'].average()
print(evoked_left)

# plot
evoked_left.plot(time_unit='s')

evoked_right = epochs['aud_r'].average()
print(evoked_right)
evoked_right.plot(time_unit='s');

There really is .A LOT. more you can do with MNE, but the techniques you'll be using obviously depend a lot on your research question and data. If you're interested in using MNE for data analysis, we can't recommend their documentation, tutorials, and example gallery enough! (See links in section X.0). Have fun! :) 